# Lab 3-1: Auto-Retraining Pipeline

## Part 3: 자동 재학습 파이프라인

이 노트북에서는 **Drift가 감지되면 자동으로 모델을 재학습**하고 배포하는 파이프라인을 구축합니다.

### 학습 목표
- Drift 감지 시 자동 재학습
- 조건부 파이프라인 실행
- MLflow에 모델 메트릭 기록

### 파이프라인 구조
```
Check Drift → Retrain Model → Deploy Model
    ↓              ↓              ↓
MLflow 조회    모델 학습       KServe 배포
(Score>0.3)   MAE: 0.39      (시뮬레이션)
```

### 핵심 개념: 조건부 분기
- Drift Score가 **0.3 이상**이면 재학습 수행
- Drift Score가 **0.3 미만**이면 재학습 건너뛰기
- 불필요한 재학습을 방지하여 리소스 절약

### 소요 시간: 약 30분

---

## Step 0: KFP SDK 설치 (필수!)

⚠️ **중요**: 아래 셀을 실행한 후 **반드시 커널을 재시작**해야 합니다.
- 메뉴: **Kernel → Restart Kernel**

In [ ]:
!pip install kfp==2.7.0 -q

print("="*60)
print("KFP SDK installed!")
print("Please restart the kernel: Kernel -> Restart Kernel")
print("="*60)

---

## Step 1: 환경 설정

⚠️ **커널 재시작 후 이 셀부터 실행하세요!**

In [ ]:
# ============================================================
# TODO: Change to your USER_NUM!
# ============================================================
USER_NUM = "01"  # ex: "01", "02", ..., "11", "20"

# Auto configuration
NAMESPACE = f"kubeflow-user{USER_NUM}"
MLFLOW_TRACKING_URI = f"http://mlflow-server.kubeflow-user{USER_NUM}.svc.cluster.local:5000"

print("=" * 60)
print("  Configuration")
print("=" * 60)
print(f"  USER_NUM: {USER_NUM}")
print(f"  NAMESPACE: {NAMESPACE}")
print(f"  MLFLOW_URI: {MLFLOW_TRACKING_URI}")
print("=" * 60)

## Step 2: 라이브러리 임포트

In [ ]:
from kfp import dsl
from kfp import compiler

import kfp
print(f"KFP Version: {kfp.__version__}")

version = kfp.__version__.split('.')
if int(version[0]) < 2 or (int(version[0]) == 2 and int(version[1]) < 7):
    print("Warning: KFP version is less than 2.7.0!")
else:
    print("KFP version OK!")

---

## Step 3: Component 1 - Drift 확인 및 재학습 결정

**MLflow에서 최근 Drift Score를 조회**하고 재학습 여부를 결정합니다.

### 로직
1. MLflow에서 가장 최근 Run의 `drift_score` 메트릭 조회
2. `drift_score > drift_threshold`이면 재학습 필요
3. 결과를 JSON으로 반환

### MLflow에서 Run 조회
```python
runs = mlflow.search_runs(
    order_by=["start_time DESC"],
    max_results=1
)
drift_score = runs.iloc[0]['metrics.drift_score']
```

In [ ]:
# MLflow URI (Pipeline parameter)
mlflow_uri_for_pipeline = MLFLOW_TRACKING_URI
print(f"MLflow URI: {mlflow_uri_for_pipeline}")

In [ ]:
@dsl.component(
    base_image="python:3.9-slim",
    packages_to_install=["mlflow==2.9.2", "pandas==2.0.3"]
)
def check_drift_and_decide(drift_threshold: float, mlflow_uri: str) -> str:
    """Check recent drift score and decide whether to retrain"""
    import json
    import os
    
    print(f"Drift Threshold: {drift_threshold}")
    
    drift_score = 0.0
    should_retrain = True  # Default: retrain if cannot check
    
    try:
        import mlflow
        
        print(f"Connecting to MLflow: {mlflow_uri}")
        os.environ['MLFLOW_TRACKING_URI'] = mlflow_uri
        mlflow.set_tracking_uri(mlflow_uri)
        
        try:
            mlflow.set_experiment("drift-monitoring")
            
            # Query recent Run
            runs = mlflow.search_runs(
                order_by=["start_time DESC"],
                max_results=1
            )
            
            if len(runs) > 0 and 'metrics.drift_score' in runs.columns:
                drift_score = float(runs.iloc[0]['metrics.drift_score'])
                should_retrain = drift_score > drift_threshold
                print(f"Found drift score from MLflow: {drift_score:.2f}")
            else:
                print("No previous runs found, defaulting to retrain")
                should_retrain = True
                
        except Exception as e:
            print(f"MLflow experiment error: {e}")
            should_retrain = True
            
    except Exception as e:
        print(f"MLflow connection failed: {e}")
        should_retrain = True
    
    result = {
        'should_retrain': bool(should_retrain),
        'drift_score': float(drift_score)
    }
    
    print(f"Decision:")
    print(f"  Drift Score: {drift_score:.2f}")
    print(f"  Should Retrain: {should_retrain}")
    
    return json.dumps(result)

print("Component 1: check_drift_and_decide defined")

---

## Step 4: Component 2 - 모델 재학습

**California Housing 데이터셋**을 사용하여 **RandomForest 모델**을 학습하고, **MLflow에 메트릭을 기록**합니다.

### 학습 파라미터
| 파라미터 | 값 |
|---------|----|
| 알고리즘 | RandomForestRegressor |
| n_estimators | 100 |
| 평가 메트릭 | MAE (Mean Absolute Error) |

### 데이터셋: California Housing
- 캘리포니아 주택 가격 예측
- 8개 특성 (MedInc, HouseAge, AveRooms 등)
- 타겟: MedHouseVal (중위 주택 가격)

In [ ]:
@dsl.component(
    base_image="python:3.9-slim",
    packages_to_install=["pandas==2.0.3", "numpy", "scikit-learn==1.3.2", "mlflow==2.9.2"]
)
def retrain_model(train_size: int, mlflow_uri: str) -> str:
    """Retrain model with California Housing dataset"""
    import pandas as pd
    import numpy as np
    from sklearn.datasets import fetch_california_housing
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_absolute_error
    import json
    import os
    import uuid
    
    print(f"Loading training data...")
    
    # Load data
    data = fetch_california_housing(as_frame=True)
    df = data.frame
    
    # Sample training data
    train_data = df.sample(n=train_size, random_state=42)
    print(f"Training data: {len(train_data)} samples")
    
    X = train_data.drop('MedHouseVal', axis=1)
    y = train_data['MedHouseVal']
    
    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    # Train model
    print("Training RandomForest model...")
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    
    # Model version
    model_version = str(uuid.uuid4())[:8]
    
    print(f"Training Results:")
    print(f"  Model version: {model_version}")
    print(f"  MAE: {mae:.4f}")
    
    # Log to MLflow
    try:
        import mlflow
        
        print(f"Connecting to MLflow: {mlflow_uri}")
        os.environ['MLFLOW_TRACKING_URI'] = mlflow_uri
        mlflow.set_tracking_uri(mlflow_uri)
        
        try:
            mlflow.set_experiment("auto-retraining")
        except:
            mlflow.create_experiment("auto-retraining")
            mlflow.set_experiment("auto-retraining")
        
        with mlflow.start_run(run_name="retrained-model"):
            mlflow.log_metric("mae", mae)
            mlflow.log_param("n_estimators", 100)
            mlflow.log_param("train_size", train_size)
            model_version = mlflow.active_run().info.run_id[:8]
        
        print("Metrics logged to MLflow successfully")
        
    except Exception as e:
        print(f"MLflow logging failed: {e}")
    
    result = {
        'model_version': str(model_version),
        'mae': float(mae),
        'status': 'trained'
    }
    
    print(f"Model trained successfully!")
    
    return json.dumps(result)

print("Component 2: retrain_model defined")

---

## Step 5: Component 3 - 모델 배포

재학습된 모델을 **배포**합니다 (시뮬레이션).

### 실제 환경에서는:
- KServe InferenceService 업데이트
- 모델 레지스트리에서 모델 가져오기
- Blue/Green 또는 Canary 배포

In [ ]:
@dsl.component(base_image="python:3.9-slim")
def deploy_model(model_result: str) -> str:
    """Deploy model simulation"""
    import json
    
    result = json.loads(model_result)
    model_version = result['model_version']
    mae = result['mae']
    
    print(f"Deploying model version: {model_version}")
    print(f"Model MAE: {mae:.4f}")
    print("Model deployed successfully!")
    
    return "deployed"

print("Component 3: deploy_model defined")

---

## Step 6: Pipeline 정의

위에서 정의한 **3개의 컴포넌트를 연결**하여 자동 재학습 파이프라인을 구성합니다.

### 파이프라인 흐름
```
1. check_task: MLflow에서 Drift Score 확인 & 재학습 결정
2. retrain_task: 모델 재학습 (check_task 완료 후 실행)
3. deploy_task: 모델 배포 (retrain_task 완료 후 실행)
```

### 실행 순서 제어: `.after()`
```python
retrain_task = retrain_model(...)
retrain_task.after(check_task)  # check_task 완료 후 retrain_task 실행
```

`.output`을 사용하지 않아도 `.after()`로 순서를 지정할 수 있습니다.

In [ ]:
@dsl.pipeline(
    name="auto-retrain",
    description="auto retraining pipeline"
)
def auto_retrain_pipeline(
    drift_threshold: float = 0.3,
    train_size: int = 5000,
    mlflow_uri: str = mlflow_uri_for_pipeline
):
    """Auto retraining pipeline"""
    
    # Step 1: Check drift and decide
    check_task = check_drift_and_decide(
        drift_threshold=drift_threshold,
        mlflow_uri=mlflow_uri
    )
    
    # Step 2: Retrain model (after check_task)
    retrain_task = retrain_model(
        train_size=train_size,
        mlflow_uri=mlflow_uri
    )
    retrain_task.after(check_task)  # Sequential execution
    
    # Step 3: Deploy model
    deploy_task = deploy_model(model_result=retrain_task.output)

print("Pipeline defined!")

---

## Step 7: Pipeline 컴파일

In [ ]:
pipeline_filename = 'auto_retrain_pipeline.yaml'

compiler.Compiler().compile(
    pipeline_func=auto_retrain_pipeline,
    package_path=pipeline_filename
)

print()
print("=" * 60)
print("  Pipeline compiled successfully!")
print("=" * 60)
print()
print(f"  Output file: {pipeline_filename}")
print()
print("  Next steps:")
print("    1. Upload pipeline to Kubeflow UI")
print("    2. Click 'Create Run'")
print("    3. Set parameters and Start")
print("=" * 60)

---

## Step 8: 생성된 YAML 파일 확인

In [ ]:
with open(pipeline_filename, 'r') as f:
    lines = f.readlines()
    print(f"File: {pipeline_filename} (first 30 lines)")
    print("=" * 60)
    for i, line in enumerate(lines[:30]):
        print(f"{i+1:3d} | {line}", end='')
    if len(lines) > 30:
        print(f"\n... ({len(lines)} lines total)")

---

## 🎯 Kubeflow UI에서 실행하기

### Step 1: Pipeline 업로드
1. Kubeflow UI → **"Pipelines"** 클릭
2. **"+ Upload pipeline"** 버튼 클릭
3. `auto_retrain_pipeline.yaml` 파일 선택
4. **"Create"** 버튼 클릭

### Step 2: Run 생성
1. 파이프라인 클릭 → **"+ Create run"** 버튼 클릭
2. Parameters 설정:
   - `drift_threshold`: 0.3
   - `train_size`: 5000
   - `mlflow_uri`: 자동 입력됨
3. **"Start"** 버튼 클릭

### Step 3: 실행 결과 확인

**Graph 탭에서 확인:**
```
✅ check-drift-and-decide: Succeeded
✅ retrain-model: Succeeded
✅ deploy-model: Succeeded
```

**예상 로그:**
```
Drift Score: 0.11
Should Retrain: False

Loading training data...
Training data: 5000 samples
Model trained successfully
Model version: db4b3de4
MAE: 0.3901

Deploying model version: db4b3de4
Model deployed successfully!
```

---

## ✅ Part 3 완료 체크리스트

- [ ] KFP SDK 2.7.0 이상 설치 완료
- [ ] USER_NUM 설정 완료
- [ ] `auto_retrain_pipeline.yaml` 파일 생성 완료
- [ ] Kubeflow UI에 파이프라인 업로드 완료
- [ ] 파이프라인 실행 성공
- [ ] 3개 컴포넌트 모두 녹색 체크 확인

---

## 📚 핵심 개념 정리

### 1. 컴포넌트 실행 순서 제어

| 방법 | 설명 | 예시 |
|------|------|------|
| `.output` | 데이터 전달 + 순서 제어 | `step2 = comp2(input=step1.output)` |
| `.after()` | 순서만 제어 (데이터 전달 X) | `step2.after(step1)` |

### 2. 재학습 전략

| 전략 | 설명 | 장점 | 단점 |
|------|------|------|------|
| **트리거 기반** | Drift 감지 시 즉시 재학습 | 빠른 대응 | 비용 높음 |
| **스케줄 기반** | 정해진 주기로 재학습 | 예측 가능 | 대응 늦음 |
| **하이브리드** | 둘 다 병행 | 균형 | 복잡성 |

### 3. MLflow 실험 조회
```python
runs = mlflow.search_runs(
    order_by=["start_time DESC"],
    max_results=1
)
drift_score = runs.iloc[0]['metrics.drift_score']
```

---

## 🎯 Lab 3-1 전체 완료!

### 학습한 내용 요약

| Part | 내용 | 핵심 기술 |
|------|------|----------|
| **Part 1** | Drift Detection | KS Test, Drift Score |
| **Part 2** | Monitoring Pipeline | Kubeflow Pipeline, MLflow |
| **Part 3** | Auto-Retraining | 순서 제어, 자동화 |

### 전체 MLOps 모니터링 사이클
```
1. 데이터 수집 → 2. Drift 감지 → 3. 알림 발송
       ↑                              ↓
6. 배포 ← 5. 모델 학습 ← 4. 재학습 결정
```

---

**수고하셨습니다! 🎉**